In [105]:
import sys
import gi
gi.require_version('Vips', '8.0')
from gi.repository import Vips

In [106]:
def gamma(band, gamma):
    return band.gamma(exponent = (1/gamma))

In [107]:
def shadows(band, lim):
    mask = (band <= lim).cast(band.BandFmt, shift = True)
    return (65535 * (band / lim * 0.1)) & mask

In [108]:
def highlights(band, lim):
    mask = (band >= lim).cast(band.BandFmt, shift = True)
    return (65535 * ((band - lim) / (65536 - lim) * 0.1 + 0.9)) & mask

In [109]:
def midtones(band, lims):
    mask = ((band > lims[0]) & (band < lims[1])).cast(band.BandFmt, shift = True)
    band = (band - lims[0]) / (lims[1] - lims[0])
    band = gamma(band, 2.15)
    return (65535 * (band * 0.8 + 0.1)) & mask    

In [110]:
def invert(band):
    band = band.invert()
    lims = (band.percent(5), band.percent(95))
    lo = shadows(band, lims[0])
    hi = highlights(band, lims[1])
    mid = midtones(band, lims)
    return (lo.add(mid).add(hi)).cast(band.BandFmt)

In [112]:
infile = "/Users/mah/Temp/2015-06-25_153918_color_ir-1.tiff"
im = Vips.Image.new_from_file(infile)
im.percent()
if im.bands >= 3:
    r = invert(im[0])
    g = invert(im[1])
    b = invert(im[2])
    im = r.bandjoin([g, b])
else:
    im = invert(im[0])        
im.write_to_file("/Users/mah/Temp/test.tiff")

In [122]:
im.alpha

Error: No such operator.
  VipsOperation: class "alpha" not found
